In [1]:
import pickle as pkl
year = 2010
DPATH = '../07_HINTS_code-main/fanxing/'
with open(f'{DPATH}/individual_data/graph_' + str(year - 5) + '_nf.pkl', 'rb') as f:
    graph_1 = pkl.load(f)
    # adj_ = [normalize(adj_)for adj_ in graph_1['adj']]  # adj: list(sp.csr_matrix)
    adj = graph_1['adj']
    # print(adj.shape)

In [2]:
print(len(adj), adj[0].shape, adj[1].shape, adj[2].shape, adj[3].shape, adj[4].shape, )
print(graph_1['feature'].shape)

5 (750602, 750602) (750602, 750602) (750602, 750602) (750602, 750602) (750602, 750602)
(750602, 4)


In [21]:
import scipy.sparse as sp
import torch
adj_coo = adj[0].tocoo()
row, col = adj_coo.row, adj_coo.col
print(len(row), len(col))
edge_index = torch.zeros((2, len(row))).to(torch.long)
edge_index[0] = torch.from_numpy(row)
edge_index[1] = torch.from_numpy(col)
print(edge_index.shape)

1058244 1058244
torch.Size([2, 1058244])


In [22]:
from torch_geometric_temporal.nn.recurrent import A3TGCN
periods = 1; inc = 4; outc=2
model = A3TGCN(in_channels=inc, out_channels=outc, periods=periods).to('cpu')
feat = torch.from_numpy(graph_1['feature']).reshape((-1, 4, 1)).to(torch.float)
a = model(feat, edge_index) # [750602, 2]
print(a.shape)

torch.Size([750602, 2])


In [23]:
from torch_geometric_temporal.nn.attention import ASTGCN
bs=1; periods = 1; inc = 4; outc=2
node_num = 10
model = ASTGCN(nb_block=2, in_channels=inc, K=2, nb_chev_filter=2, nb_time_filter=2, time_strides=1,\
    num_for_predict=outc, len_input=periods, num_of_vertices=node_num).to('cuda')
# feat = torch.from_numpy(graph_1['feature']).reshape((-1, 4, 1)).to(torch.float)
feat = torch.rand(bs, node_num, 4, 1).to('cuda')
edge_index = (torch.rand(2, 5)*node_num).to(torch.long).to('cuda')
a = model(feat, edge_index) # [750602, 2]
print(a.shape)

torch.Size([1, 10, 2])


In [26]:
from torch_geometric_temporal.nn.attention import MSTGCN
bs=1; periods = 1; inc = 4; outc=2
node_num = 10
model = MSTGCN(nb_block=2, in_channels=inc, K=2, nb_chev_filter=2, nb_time_filter=2, time_strides=1,\
    num_for_predict=outc, len_input=periods).to('cuda')
feat = torch.from_numpy(graph_1['feature']).reshape((bs, -1, 4, 1)).to(torch.float)
# feat = torch.rand(bs, node_num, 4, 1).to('cuda')
# edge_index = (torch.rand(2, 5)*node_num).to(torch.long).to('cuda')
a = model(feat.to('cuda'), edge_index.to('cuda')) # [750602, 2]
print(a.shape)

torch.Size([1, 750602, 2])


In [18]:
bs=1; periods = 1; inc = 4; outc=2
node_num = len(graph_1['feature'])
model = ASTGCN(nb_block=1, in_channels=inc, K=1, nb_chev_filter=1, nb_time_filter=1, time_strides=1,\
    num_for_predict=outc, len_input=periods, num_of_vertices=node_num).to('cuda')
feat = torch.from_numpy(graph_1['feature']).reshape((bs, -1, 4, 1)).to(torch.float).to('cuda')
# feat = torch.rand(bs, node_num, 4, 1).to('cuda')
# edge_index = (torch.rand(2, 5)*node_num).to(torch.long).to('cuda')
a = model(feat, edge_index.to('cuda')) # [750602, 2]
print(a.shape)

RuntimeError: [enforce fail at CPUAllocator.cpp:67] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 2253613449616 bytes. Error code 12 (Cannot allocate memory)

In [28]:
a[rank].shape

torch.Size([3000, 2])

In [ ]:

model = TGCN(in_channels=inc, out_channels=outc)
feat = torch.from_numpy(graph_1['feature']).reshape((-1, 4, 1)).to(torch.float)
a = model(feat, edge_index)

In [17]:
# import torch.nn as nn
# loss_f = nn.MSELoss()
rank = pkl.load(open(f'../07_HINTS_code-main/aminer/select_index_train.pkl', 'rb')) # [3000]
labels = pkl.load(open(f'{DPATH}/data/cumulative_log_labels_new.pkl', 'rb'))['P' + str(year) + '_label'].iloc[rank, 1:6].values
print(rank.shape, labels.shape)


(3000,) (3000, 5)


In [18]:
out = a[rank]
out.shape

# print(out.shape, lab.shape)

torch.Size([3000, 2])

In [7]:
index = pkl.load(open(f'{DPATH}/individual_data/index_' + str(year) + '.pkl', 'rb')) 
index_table_P1P = index[0]
print(index[0].shape)

(5, 189098, 100)


In [8]:
id_item, item_id = graph_1['id_item'], graph_1['item_id']
print(len(id_item), len(item_id))

750602 750602


In [9]:
item_id['53e99abeb7602d970233cdda']

KeyError: '53e99abeb7602d970233cdda'

In [ ]:
# from tqdm import trange
# for i in trange(len(id_item)):
#     item = id_item[i]
#     df3 = labels[labels['P']==item]
#     if len(df3)>0:
#         print(i, item)
#         print(df3)
#         break



  0%|          | 771/750602 [00:14<3:58:27, 52.41it/s]

771 53e9b746b7602d97042e3386
                               P  citation_2011  citation_2012  citation_2013  \
161483  53e9b746b7602d97042e3386            0.0            0.0            0.0   

        citation_2014  citation_2015  citation_2016  citation_2017  \
161483            0.0            0.0            0.0            0.0   

        citation_2018  citation_2019  citation_2020  citation_2021  
161483            0.0            0.0            0.0            0.0  


(3000,)

In [11]:
index_table_P1P = index[0][:,rank,:]
print(index_table_P1P.shape)
# batch_size = 2
# index_P = [index[i*batch_size:(i+1)*batch_size,:] for index in index_table_P1P]

(5, 3000, 100)


In [12]:
rank[0:10]

array([ 2,  7,  9, 11, 17, 23, 24, 25, 28, 31])

In [13]:
embed = a[index_table_P1P[0]]
embed.shape

torch.Size([3000, 100, 2])

In [25]:
import numpy as np
def lable_normorlization(labels):
    maximum = labels.max()
    minimum = labels.min()
    new_value = (labels-minimum)/(maximum-minimum)
    return new_value,maximum,minimum


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1)) # [n, 1]
    r_inv = np.power(rowsum, -1).flatten() #[n]
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv) # [n, n]
    mx = r_mat_inv.dot(mx) # [n, n]
    return mx


def adj2edge_index(adj):
    adj_coo = adj.tocoo()
    row, col = adj_coo.row, adj_coo.col
    edge_index = torch.zeros((2, len(row))).to(torch.long)
    edge_index[0] = torch.from_numpy(row)
    edge_index[1] = torch.from_numpy(col)
    return edge_index


# def load_data_dblpV13(year,flag):
def load_data_aminer(year,flag):
    # DPATH = '../07_HINTS_code-main/fanxing'
    DPATH = '../07_HINTS_code-main/aminer'
    with open(f'{DPATH}/individual_data/graph_' + str(year - 5) + '_nf.pkl', 'rb') as f:
        graph_1 = pkl.load(f)
        adj_ = [normalize(adj_)for adj_ in graph_1['adj']] # adj: list(sp.csr_matrix), 5个类型
        adj_1 = [adj2edge_index(adj) for adj in adj_]
        feature_1 = graph_1['feature'] # np.array, [n, 4]

    with open(f'{DPATH}/individual_data/graph_' + str(year - 4) + '_nf.pkl', 'rb') as f:
        graph_2 = pkl.load(f)
        adj_ = [normalize(adj_)for adj_ in graph_2['adj']]
        adj_2 = [adj2edge_index(adj) for adj in adj_]
        feature_2 = graph_2['feature']
    
    with open(f'{DPATH}/individual_data/graph_' + str(year - 3) + '_nf.pkl', 'rb') as f:
        graph_3 = pkl.load(f)
        adj_ = [normalize(adj_)for adj_ in graph_3['adj']]
        adj_3 = [adj2edge_index(adj) for adj in adj_]
        feature_3 = graph_3['feature']

    with open(f'{DPATH}/individual_data/graph_' + str(year - 2) + '_nf.pkl', 'rb') as f:
        graph_4 = pkl.load(f)
        adj_ = [normalize(adj_)for adj_ in graph_4['adj']]
        adj_4 = [adj2edge_index(adj) for adj in adj_]
        feature_4 = graph_4['feature']

    with open(f'{DPATH}/individual_data/graph_' + str(year - 1) + '_nf.pkl', 'rb') as f:
        graph_5 = pkl.load(f)
        adj_ = [normalize(adj_)for adj_ in graph_5['adj']]
        adj_5 = [adj2edge_index(adj) for adj in adj_]
        feature_5 = graph_5['feature']
    
    edge_list = []
    for i in range(len(adj_1)):
        # tt = torch.stack([adj_1[0], adj_2[1], adj_3,adj_4,adj_5], dim=)
        edge_list.append( [adj_1[i], adj_2[i], adj_3[i], adj_4[i], adj_5[i] ] )

    print(feature_1.shape, feature_2.shape, feature_3.shape, feature_4.shape, feature_5.shape, )
    ls = (600391, 4)
    # for i in range(2):
    #     ls[i] = min(feature_1.shape[i], feature_2.shape[i], feature_3.shape[i], feature_4.shape[i], feature_5.shape[i])
    # ee = min(feature_1.shape[1], feature_2.shape[0], feature_3.shape[0], feature_4.shape[0], feature_5.shape[0])
    feature_np = np.stack([feature_1[:ls[0], :ls[1]], feature_2[:ls[0], :ls[1]], feature_3[:ls[0], :ls[1]],\
        feature_4[:ls[0], :ls[1]], feature_5[:ls[0], :ls[1]] ], axis=-1) # [n, 4, 5]
    feature = torch.from_numpy(feature_np).to(torch.float)

    if flag == "train":
        with open(f'{DPATH}/select_index_train.pkl','rb') as f:
            rank = pkl.load(f) # rank : shape=(3000,), [   2    7    9 ... 9009 9045 9062]

    elif flag == "test":
        with open(F'{DPATH}/select_index_test.pkl','rb') as f:
            rank = pkl.load(f)
    
    with open(f'{DPATH}/cumulative_log_labels_new.pkl','rb') as f: # len=13, 2005-2017的引用情况
        # 13个csv，每一列是year-2018的引用量
        labels = pkl.load(f)['P' + str(year) + '_label'].iloc[rank, 1:6].values # shape=(3000, 5)
    
    labels, label_max, label_min = lable_normorlization(labels)

    return edge_list, feature, labels, label_max, label_min

edge_list, feature, labels, label_max, label_min = load_data_aminer(2010, 'train')

/home/ztl/miniconda3/envs/expc/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in power
  if sys.path[0] == '':


(600391, 4) (697400, 4) (789677, 4) (867984, 4) (977953, 4)


In [23]:
edge_list[0]

[tensor([], size=(2, 0), dtype=torch.int64),
 tensor([], size=(2, 0), dtype=torch.int64),
 tensor([], size=(2, 0), dtype=torch.int64),
 tensor([], size=(2, 0), dtype=torch.int64),
 tensor([], size=(2, 0), dtype=torch.int64)]

In [26]:
print(labels.shape)

(3000, 5)
